In [6]:
from transformers import AutoModel, AutoProcessor, SpeechT5HifiGan, SpeechT5ForTextToSpeech, SpeechT5Processor
from speechbrain.pretrained import EncoderClassifier
from datasets import load_dataset 
import torch 
import text_to_speech 
import sounddevice
import torchaudio
import librosa
import torch.nn.functional as F 

In [7]:
processor = SpeechT5Processor.from_pretrained("../../hf_models/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("../../hf_models/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("../../hf_models/speecht5_hifigan")
classifier = EncoderClassifier.from_hparams(source="../../hf_models/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

In [10]:
model.speecht5.decoder.prenet

SpeechT5SpeechDecoderPrenet(
  (layers): ModuleList(
    (0): Linear(in_features=80, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (final_layer): Linear(in_features=256, out_features=768, bias=True)
  (encode_positions): SpeechT5ScaledPositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (speaker_embeds_layer): Linear(in_features=1280, out_features=768, bias=True)
)

In [177]:
# embedding_dataset = load_dataset(path='../../data/cmu-arctic-xvectors', split='validation')
# embeddings = torch.tensor(embedding_dataset[7306]["xvector"]).unsqueeze(0)
signal, fs = torchaudio.load('rose_.mp3')
assert fs == 16000
embeddings = classifier.encode_batch(signal[:, 16000*0:16000*9]).mean(dim=0)

# if speaker_embeddings is not None:
#   speaker_embeddings = nn.functional.normalize(speaker_embeddings)
#   speaker_embeddings = speaker_embeddings.unsqueeze(1)
#   speaker_embeddings = speaker_embeddings.expand(-1, inputs_embeds.size(1), -1)
#   inputs_embeds = torch.cat([inputs_embeds, speaker_embeddings], dim=-1)
#   inputs_embeds = nn.functional.relu(self.speaker_embeds_layer(inputs_embeds))

# return inputs_embeds

In [183]:
# speak = 'Roseanne Park, born on eleventh February, nineteen ninety seven, known mononymously as Rosé, is a Korean-New Zealander singer and dancer based in South Korea. Born in New Zealand and raised in Australia, Rosé signed with South Korean label Y G Entertainment following a successful audition in twenth twelve and trained for four years before debuting as a member of the girl group Blackpink in August twenty sixteen.'
# speak = 'When I go out and there are hundreds of people taking photographs of me.'
speak = 'No, thats not how you do it'
speech = text_to_speech.tts(processor, model, vocoder, speak, embeddings)
sounddevice.play(speech.numpy(), 16000)

In [5]:
embeddings.shape 

torch.Size([2, 1, 192])